In [98]:
## Exercise: Let's explore our local weather forecast.
## 
## Scrape your local weather forecast.
## What is today's weather like?
## Extract the longer weather description for each forecast.
## Extract the numeric temperature for all the forecasts.
## Store the Day, Short Forecast, Numeric Temperature, High or Low, and extended forecast in a data structure 
## of your choice. Export your final data to a pickle object.
## Extra challenge: Build a program that can extract the extended forecast from the 3 largest cities near you.
## Data: forecast.weather.gov
## Deliverable: Python File or Notebook

In [225]:
import requests
from bs4 import BeautifulSoup as bs

url = 'http://forecast.weather.gov/MapClick.php?lat=47.6062&lon=-122.3321#.WHmYnKIrJE4'
html_doc = requests.get(url).content
print (html_doc[:200])

soup = bs(html_doc, 'html.parser')
print(soup.prettify()[:200])


b'<!DOCTYPE html>\n<html class="no-js">\n    <head>\n        <!-- Meta -->\n        <meta name="viewport" content="width=device-width">\n        <link rel="schema.DC" href="http://purl.org/dc/elements/1.1/" '
<!DOCTYPE html>
<html class="no-js">
 <head>
  <!-- Meta -->
  <meta content="width=device-width" name="viewport">
   <link href="http://purl.org/dc/elements/1.1/" rel="schema.DC"/>
   <title>
    Nat


In [226]:
detailed_fc = soup.find(attrs={'id':'detailed-forecast-body'})
print (detailed_fc.prettify()[:400])

<div class="panel-body" id="detailed-forecast-body">
 <div class="row row-odd row-forecast">
  <div class="col-sm-2 forecast-label">
   <b>
    Tonight
   </b>
  </div>
  <div class="col-sm-10 forecast-text">
   Mostly cloudy, with a low around 29. Calm wind becoming east northeast around 6 mph after midnight.
  </div>
 </div>
 <div class="row row-even row-forecast">
  <div class="col-sm-2 forecas


In [238]:
detailed_fc_parsed = []
for row in detailed_fc.find_all(class_='row-forecast'):
    detailed_fc_parsed.append([row.find(class_='forecast-label').get_text().strip(),
                               row.find(class_='forecast-text').get_text().strip()])
print(detailed_fc_parsed[:2])


[['Tonight', 'Mostly cloudy, with a low around 29. Calm wind becoming east northeast around 6 mph after midnight.'], ['Sunday', 'Mostly sunny, with a high near 41. Southeast wind around 5 mph becoming calm  in the morning.']]


In [227]:
extended_fc = soup.find(attrs={'id':'seven-day-forecast-container'})
print (extended_fc.prettify()[:800])

<div id="seven-day-forecast-container">
 <ul class="list-unstyled" id="seven-day-forecast-list">
  <li class="forecast-tombstone">
   <div class="tombstone-container">
    <p class="period-name">
     Tonight
     <br>
      <br/>
     </br>
    </p>
    <p>
     <img alt="Tonight: Mostly cloudy, with a low around 29. Calm wind becoming east northeast around 6 mph after midnight. " class="forecast-icon" src="newimages/medium/nbkn.png" title="Tonight: Mostly cloudy, with a low around 29. Calm wind becoming east northeast around 6 mph after midnight. "/>
    </p>
    <p class="short-desc">
     Mostly Cloudy
    </p>
    <p class="temp temp-low">
     Low: 29 °F
    </p>
   </div>
  </li>
  <li class="forecast-tombstone">
   <div class="tombstone-container">
    <p class="period-name">
     


In [237]:
from PIL import Image
from io import BytesIO
# god bless stackoverflow...

extended_fc_parsed = []

for i, container in enumerate(extended_fc.find_all(class_='tombstone-container')):
    extended_fc_parsed.append({})
    extended_fc_parsed[i]['period-name'] = container.find(class_='period-name').get_text()
    extended_fc_parsed[i]['fc-short'] = container.find(class_='short-desc').get_text()
    temp_string = container.find(class_='temp').get_text().strip()
    extended_fc_parsed[i]['temp'] = int(temp_string[temp_string.find(':')+1:-2].strip())
    extended_fc_parsed[i]['temp-type'] = temp_string[:temp_string.find(':')]
    extended_fc_parsed[i]['temp-uom'] = container.find(class_='temp').get_text()[-1]
    extended_fc_parsed[i]['img-src'] = container.img.attrs['src']
    extended_fc_parsed[i]['fc-detailed'] = container.img.attrs['title'].strip()
    response = requests.get('http://forecast.weather.gov/' + container.img.attrs['src'])
    extended_fc_parsed[i]['image'] = Image.open(BytesIO(response.content))
            
print (extended_fc_parsed[:2])

[{'temp-uom': 'F', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=86x86 at 0x107D4E7B8>, 'temp': 29, 'temp-type': 'Low', 'period-name': 'Tonight', 'fc-detailed': 'Tonight: Mostly cloudy, with a low around 29. Calm wind becoming east northeast around 6 mph after midnight. ', 'img-src': 'newimages/medium/nbkn.png', 'fc-short': 'Mostly Cloudy'}, {'temp-uom': 'F', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=86x86 at 0x1076E5668>, 'temp': 41, 'temp-type': 'High', 'period-name': 'Sunday', 'fc-detailed': 'Sunday: Mostly sunny, with a high near 41. Southeast wind around 5 mph becoming calm  in the morning. ', 'img-src': 'newimages/medium/sct.png', 'fc-short': 'Mostly Sunny'}]


In [233]:
import pickle
pickle_detailed_fc = pickle.dumps(detailed_fc_parsed)
pickle_extended_fc = pickle.dumps(extended_fc_parsed)

In [ ]:
## YZ Question: OK, so I dumped my data into pickle project. What's the practical use? Thanks.